In [6]:
# evaluation/evaluator.py
import os
import json
import time
import tempfile
import subprocess
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer

class CodeEvaluator:
    def __init__(self, custom_model, copilot_model, output_dir="../results"):
        """Initialize the code evaluator"""
        self.custom_model = custom_model
        self.copilot_model = copilot_model
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize rouge scorer for text similarity
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    def functional_correctness(self, code, test_code):
        """Evaluate if the code passes the test cases"""
        # Create a temporary file to run the code
        with tempfile.NamedTemporaryFile(suffix='.py', delete=False) as f:
            f.write((code + '\n\n' + test_code).encode('utf-8'))
            temp_file = f.name
        
        try:
            # Run the test code
            result = subprocess.run(
                ['python', temp_file],
                capture_output=True,
                text=True,
                timeout=5  # Limit execution time to avoid infinite loops
            )
            
            # Check if tests passed
            success = result.returncode == 0
            return {
                'success': success,
                'stdout': result.stdout,
                'stderr': result.stderr
            }
            
        except subprocess.TimeoutExpired:
            return {
                'success': False,
                'stdout': '',
                'stderr': 'Execution timed out'
            }
        except Exception as e:
            return {
                'success': False,
                'stdout': '',
                'stderr': str(e)
            }
        finally:
            # Clean up
            if os.path.exists(temp_file):
                os.remove(temp_file)
    
    def measure_code_complexity(self, code):
        """Measure code complexity metrics"""
        # Simple complexity metrics
        line_count = len(code.strip().split('\n'))
        char_count = len(code)
        
        # Count control structures as a rough proxy for cyclomatic complexity
        control_keywords = [
            'if ', 'else:', 'elif ', 'for ', 'while ', 'try:', 'except', 
            'with ', 'def ', 'class '
        ]
        
        control_count = sum(code.count(keyword) for keyword in control_keywords)
        
        return {
            'line_count': line_count,
            'char_count': char_count,
            'control_count': control_count
        }
    
    def measure_similarity(self, generated_code, reference_code):
        """Measure similarity between generated code and reference solution"""
        # Use ROUGE scores for similarity
        scores = self.scorer.score(reference_code, generated_code)
        
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }
    
    def measure_generation_time(self, model, prompt):
        """Measure code generation time"""
        start_time = time.time()
        _ = model.generate_code(prompt)
        end_time = time.time()
        
        return end_time - start_time
    
    def evaluate_models(self, test_dataset_path="../data/human_eval_dataset.json", num_samples=10):
        """Evaluate and compare both models on the test dataset"""
        # Load test dataset
        with open(test_dataset_path, 'r') as f:
            test_data = json.load(f)
        
        # Limit samples if needed
        test_data = test_data[:num_samples]
        
        results = []
        
        for i, item in enumerate(test_data):
            print(f"Evaluating sample {i+1}/{len(test_data)}")
            prompt = item['prompt']
            reference_solution = item['canonical_solution']
            test_code = item['test']
            
            # Generate code with both models
            custom_code = self.custom_model.generate_code(prompt)
            copilot_code = self.copilot_model.generate_code(prompt)
            
            # Measure generation time
            custom_time = self.measure_generation_time(self.custom_model, prompt)
            copilot_time = self.measure_generation_time(self.copilot_model, prompt)
            
            # Evaluate functional correctness
            custom_correctness = self.functional_correctness(
                custom_code, test_code
            )
            copilot_correctness = self.functional_correctness(
                copilot_code, test_code
            )
            
            # Measure code complexity
            custom_complexity = self.measure_code_complexity(custom_code)
            copilot_complexity = self.measure_code_complexity(copilot_code)
            
            # Measure similarity to reference solution
            custom_similarity = self.measure_similarity(custom_code, reference_solution)
            copilot_similarity = self.measure_similarity(copilot_code, reference_solution)
            
            # Store results
            results.append({
                'task_id': item['task_id'],
                'prompt': prompt,
                'custom_code': custom_code,
                'copilot_code': copilot_code,
                'reference_solution': reference_solution,
                
                'custom_success': custom_correctness['success'],
                'copilot_success': copilot_correctness['success'],
                
                'custom_time': custom_time,
                'copilot_time': copilot_time,
                
                'custom_line_count': custom_complexity['line_count'],
                'copilot_line_count': copilot_complexity['line_count'],
                'custom_control_count': custom_complexity['control_count'],
                'copilot_control_count': copilot_complexity['control_count'],
                
                'custom_rouge1': custom_similarity['rouge1'],
                'copilot_rouge1': copilot_similarity['rouge1'],
                'custom_rougeL': custom_similarity['rougeL'],
                'copilot_rougeL': copilot_similarity['rougeL'],
            })
        
        # Save detailed results
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(self.output_dir, 'detailed_results.csv'), index=False)
        
        # Compute and save summary metrics
        summary = {
            'custom_success_rate': results_df['custom_success'].mean(),
            'copilot_success_rate': results_df['copilot_success'].mean(),
            
            'custom_avg_time': results_df['custom_time'].mean(),
            'copilot_avg_time': results_df['copilot_time'].mean(),
            
            'custom_avg_lines': results_df['custom_line_count'].mean(),
            'copilot_avg_lines': results_df['copilot_line_count'].mean(),
            
            'custom_avg_rouge1': results_df['custom_rouge1'].mean(),
            'copilot_avg_rouge1': results_df['copilot_rouge1'].mean(),
            'custom_avg_rougeL': results_df['custom_rougeL'].mean(),
            'copilot_avg_rougeL': results_df['copilot_rougeL'].mean(),
        }
        
        summary_df = pd.DataFrame([summary])
        summary_df.to_csv(os.path.join(self.output_dir, 'summary_results.csv'), index=False)
        
        return results_df, summary_df